In [2]:
# Import library for load data
import ray
ray.init(num_cpus=8)
import modin.pandas as pd
#import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
import seaborn as sns
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from category_encoders import OrdinalEncoder
from numpy import genfromtxt
import sys
import csv
# Import deeplearning library
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms 
from torch.utils.data import Dataset
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np 

2023-07-18 01:29:52,848	INFO worker.py:1636 -- Started a local Ray instance.


In [3]:
# The columns name of dataset is contain space, this function will strip it out (data = fix_columns_name(data))
def fix_columns_name(data):
    data.columns = [column_name.strip() for column_name in data.columns]
    return data

# Drop all string data in dataset (data = drop_non_numberic_columns(data))
def drop_non_numberic_columns(data):
    data.drop(['Unnamed: 0', 'Flow ID','Source IP','Destination IP', 'Timestamp', 'SimillarHTTP'], inplace=True, axis=1)
    return data

#Create a copy of pandas frame with dropped NaN and Inf value (data = drop_NaN(data))
def drop_NaN(data):
    dropped = data.copy(deep=True)
    dropped.replace([np.inf, -np.inf], np.nan, inplace=True)
    dropped.dropna(inplace=True)
    return dropped

# Return a dictionary of available labels and it number from dataset 'Label' column (labels_dict = available_lable(data))
# Input: dataset
# Output: 
# Traning dataset: {'Syn': 0, 'DrDoS_SNMP': 1, 'DrDoS_UDP': 2, 'DrDoS_DNS': 3, 'TFTP': 4, 'DrDoS_LDAP': 5, 'UDP-lag': 6, 'DrDoS_NTP': 7, 'DrDoS_MSSQL': 8, 'DrDoS_NetBIOS': 9, 'BENIGN': 10, 'WebDDoS': 11, 'DrDoS_SSDP': 12}
# Testing dataset {'Syn': 0, 'NetBIOS': 1, 'UDPLag': 2, 'LDAP': 3, 'MSSQL': 4, 'BENIGN': 5, 'Portmap': 6, 'UDP': 7} 
def available_label(data):
    data_labels = set(data['Label'])
    labels_dict = {}
    for i, label in enumerate(data_labels):
        labels_dict.update({label:i})
    return labels_dict

# Caculate flow labels number and return dictionary of {labels : count}
# train_flow_labels_count = flow_labels_count(data)
def flows_labels_count(data):
    labels = {}
    for flow_label in data['Label']:
        if flow_label in labels.keys():
            labels[flow_label] += 1
        else:
            labels[flow_label] = 0
    return labels

# Encoding labels to assigned number for learning
# data = encoding_lables('train', data)
def encoding_labels(dataset_type, data):
    if dataset_type == 'train':
        labels_num = {'DrDoS_DNS': 0, 'DrDoS_UDP': 1, 'TFTP': 2, 'Syn': 3, 'DrDoS_MSSQL': 4, 'UDP-lag': 5, 'DrDoS_NetBIOS': 6, 'DrDoS_NTP': 7, 'WebDDoS': 8, 'DrDoS_SSDP': 9, 'BENIGN': 10, 'DrDoS_SNMP': 11, 'DrDoS_LDAP': 12}
    elif dataset_type == 'test':
        labels_num = {'Syn': 3, 'NetBIOS': 6, 'UDPLag': 5, 'LDAP': 12, 'MSSQL': 4, 'BENIGN': 10, 'Portmap': 13, 'UDP': 1} 
    for label, number in labels_num.items():
        data['Label'] = data['Label'].replace(label, number)
    return data

# Ploting data
def data_plot(data):
    data_flow_labels_count = flows_labels_count(data)
    for flow_label, flow_number in data_flow_labels_count.items():
        print('Number of {} flow is: {}/{} ({:.2f})'.format(flow_label, flow_number, sum(data_flow_labels_count.values()), flow_number/sum(data_flow_labels_count.values())*100))
    #Create Flow Number based on label Figure
    data_flows_figure_num = plt.figure(figsize = (20, 5))
    # creating the bar plot
    plt.bar(data_flow_labels_count.keys(), data_flow_labels_count.values(), color ='red',
            width = 0.5)
    plt.xlabel("Flows Label")
    plt.ylabel("Number of FLows")
    plt.title("Flows label and number")
    plt.show()

In [5]:
print('-------------ANALYZE DATASET-------------')
#Load dataset
print("Loading dataset, please wait .....")
train = pd.read_csv('/home/jun/CICIDS2019/TrainCIC2019.csv')
#test = pd.read_csv('/home/jun/CICIDS2019/TestCIC2019.csv')
print("DONE!")
print("-------------------------")

# Fix columns name
print("Start fixing columns name")
train = fix_columns_name(train)
print("DONE!")
print("-------------------------")

# Drop NaN and inf
print("Start dropping NaN and Infinity contained row")
train = drop_NaN(train)
print("DONE!")
print("-------------------------")

# Encoding train dataset label
print("Start encoding flow labels to number")
train = encoding_labels('train', train)
print("DONE!")
print("-------------------------")

# Drop num numberic columns ('Flow ID','Source IP','Destination IP', 'Timestamp', 'SimillarHTTP')
print("Dropping ('Flow ID','Source IP','Destination IP', 'Timestamp', 'SimillarHTTP')")
train = drop_non_numberic_columns(train)
print("DONE!")
print("-------------------------")

#Shuffle data for training 
print("Shuffling data")
train = train.sample(frac=1)
print("DONE!")
print("-------------------------")

# Change data to numpy
print("Change data into numpy")
train = train.to_numpy()
train = np.asfarray(train)
print("DONE!")
print("-------------------------")

-------------ANALYZE DATASET-------------
Loading dataset, please wait .....
DONE!
-------------------------
Start fixing columns name
DONE!
-------------------------
Start dropping NaN and Infinity contained row
DONE!
-------------------------
Start encoding flow labels to number
DONE!
-------------------------
Dropping ('Flow ID','Source IP','Destination IP', 'Timestamp', 'SimillarHTTP')
DONE!
-------------------------
Shuffling data


(raylet) Spilled 2929 MiB, 3 objects, write throughput 741 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 5757 MiB, 11 objects, write throughput 803 MiB/s.


DONE!
-------------------------
Change data into numpy


(raylet) Spilled 10512 MiB, 23 objects, write throughput 940 MiB/s.
(raylet) Spilled 16907 MiB, 31 objects, write throughput 953 MiB/s.
(raylet) Spilled 73747 MiB, 119 objects, write throughput 2380 MiB/s.
(raylet) Spilled 77436 MiB, 122 objects, write throughput 2413 MiB/s.


DONE!
-------------------------


In [6]:
# Split dataset to data and label
train_day_X = train[:, :81]
train_day_y = train[:, 81]

In [7]:
# Normalize data to range (0, 1)
scaler = MinMaxScaler()
scaler.fit(train_day_X)
train_normalized = scaler.transform(train_day_X)

# Slit training day data to train (80%), val (0.0002) and test (0.1998)
X_train, X_val_test, y_train, y_val_test = train_test_split(train_day_X, train_day_y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.99, random_state=42) 

In [8]:
# Define Deeplearning parameters
batch_size = 10000
learning_rate = 0.001
num_epochs = 100

In [9]:
# Define CUDA device 
device = torch.device(3 if torch.cuda.is_available() else 'cpu')

In [10]:
# Custom dataset class
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.from_numpy(data)
        self.labels = torch.from_numpy(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

Train_dataset = MyDataset(X_train.astype(np.float32), y_train.astype(np.float32))
Val_dataset = MyDataset(X_val.astype(np.float32), y_val.astype(np.float32))
Test_dataset = MyDataset(X_test.astype(np.float32), y_test.astype(np.float32))

# Create a data loader for batch training
train_loader = torch.utils.data.DataLoader(Train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(Val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(Test_dataset, batch_size=batch_size, shuffle=False)

# Neural Model
# ANN (inputs_size = 81, 128, 256, 512, 256, 128, 13)
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(81, 128)
        self.l2 = nn.Linear(128, 256)
        self.l3 = nn.Linear(256, 512)
        self.l4 = nn.Linear(512, 256)
        self.l5 = nn.Linear(256, 128)
        self.l6 = nn.Linear(128, 13)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.25)
    
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.l4(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.l5(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.l6(out)
        return out
    
def get_test_loss_acc(net, criterion, data_loader):
    """A simple function that iterates over `data_loader` to calculate the overall loss"""
    testing_loss = []
    testing_acc = []
    avg_loss = 0
    avg_acc = 0
    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data
            # get the data to GPU (if available)
            inputs, labels = inputs.to(device), labels.to(device, dtype = torch.long) # Change to data type long to not rise error
            outputs = net(inputs)
            # calculate the loss for this batch
            loss = criterion(outputs, labels)
            # add the loss of this batch to the list
            testing_loss.append(loss.item())
            _, predicted = torch.max(outputs, 1) # Get prediction on batch (shape: batch_size)
            batch_correct_predicted = (predicted == labels).sum().item() # Get number of true prediction on batch
            testing_acc.append(batch_correct_predicted/batch_size*100) # Append prediction accuracy of this batch
    avg_loss = np.mean(testing_loss) 
    avg_acc = np.mean(testing_acc) 
    
    # calculate the average loss
    return {'loss': avg_loss, 'acc': avg_acc}

# Define model, put into GPU
model = NeuralNet()
model.to(device)

NeuralNet(
  (l1): Linear(in_features=81, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=512, bias=True)
  (l4): Linear(in_features=512, out_features=256, bias=True)
  (l5): Linear(in_features=256, out_features=128, bias=True)
  (l6): Linear(in_features=128, out_features=13, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.25, inplace=False)
)

In [11]:
# Define model, put into GPU
model = NeuralNet()
model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
writer = SummaryWriter()

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

# Create csv file to plot data
f = open('/home/jun/CICIDS2019/csv/ANN3.csv', 'w')
writer = csv.writer(f)
header = ['10000 batch', 'train_loss', 'train_acc', 'test_loss', 'test_acc']
writer.writerow(header)

53

In [12]:
# Training loop
train_loss_d, train_acc_d, test_loss_d, test_acc_d = [], [], [], []
running_loss = []
running_acc = []
n_total_step = len(train_loader)
for epoch in range(num_epochs):
    # Total loss on all training sample
    total_loss = []
    # Total accuracy on all training sample
    total_acc = []
    for i, (data, labels) in enumerate(train_loader):
        # Batch size: 10000 
        # 1000000, 81
        labels = labels.to(device, dtype = torch.long) # Change labels to long data type for cross entropy
        data = data.to(device)
        
        # forward 
        outputs = model(data)
        loss = criterion(outputs, labels)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Batch loss
        running_loss.append(loss.item())

        # Batch accuracy calculate
        _, predicted = torch.max(outputs, 1) # Get prediction on batch (shape: batch_size)
        batch_correct_predicted = (predicted == labels).sum().item() # Get number of true prediction on batch
        running_acc.append(batch_correct_predicted/batch_size*100) # Append prediction accuracy 
        if i % 100 == 0:
            # Calculate average train loss on 10000 batch
            avg_train_loss = np.mean(running_loss)
            running_loss.clear()
            # Calculate average train accuracy on 10000 batch
            avg_train_acc = np.mean(running_acc)
            running_acc.clear()
            # Adding batch loss to total loss
            # Calculate average test loss on all val dataset
            avg_test_loss = get_test_loss_acc(model, criterion, val_loader)['loss']
            # Calculate average train accuracy on all val dataset
            avg_test_acc = get_test_loss_acc(model, criterion, val_loader)['acc']
            total_loss.append(avg_train_loss)
            # Adding batch accuracy to total accuracy
            total_acc.append(avg_train_acc)
            # Adding data to plot after training
            train_loss_d.append(avg_train_loss) 
            train_acc_d.append(avg_train_acc) 
            test_loss_d.append(avg_test_loss) 
            test_acc_d.append(avg_test_acc) 
            print(f'epoch {epoch + 1}/{num_epochs}, step [{i+1}/{n_total_step}], train_loss = {avg_train_loss:.3f}, train_acc = {avg_train_acc:.3f}, test_loss = {avg_test_loss:.3f}, test_acc = {avg_test_acc:.3f}')
            # Write plot data to csv file
            plot_data = [i+1,avg_train_loss,avg_train_acc,avg_test_loss,avg_test_acc]
            writer.writerow(plot_data)
    scheduler.step()
    print('Epoch: [{}/{}], train_Loss: {:.3f}, train_accuracy: {:.3f}'.format(epoch + 1, num_epochs, np.mean(total_loss), np.mean(total_acc)))
    PATH = '/home/jun/CICIDS2019/saved_model/ANN3' + str(epoch+1) + 'epoch.pth'
    torch.save(model.state_dict(), PATH)

epoch 1/100, step [1/3409], train_loss = 1186813.500, train_acc = 5.410, test_loss = 1159653.925, test_acc = 19.215
epoch 1/100, step [101/3409], train_loss = 49924.117, train_acc = 28.823, test_loss = 200.588, test_acc = 31.943
epoch 1/100, step [201/3409], train_loss = 67.622, train_acc = 39.773, test_loss = 9.874, test_acc = 40.527
epoch 1/100, step [301/3409], train_loss = 9.109, train_acc = 42.000, test_loss = 4.649, test_acc = 41.075
epoch 1/100, step [401/3409], train_loss = 7.262, train_acc = 42.313, test_loss = 3.410, test_acc = 41.253
epoch 1/100, step [501/3409], train_loss = 4.323, train_acc = 42.437, test_loss = 2.245, test_acc = 41.423
epoch 1/100, step [601/3409], train_loss = 3.098, train_acc = 42.522, test_loss = 2.224, test_acc = 41.337
epoch 1/100, step [701/3409], train_loss = 3.050, train_acc = 42.692, test_loss = 3.547, test_acc = 41.487
epoch 1/100, step [801/3409], train_loss = 2.718, train_acc = 42.824, test_loss = 1.891, test_acc = 41.601
epoch 1/100, step [90

KeyboardInterrupt: 

In [13]:
# Set batch size for training
batch_size = 1000000

# Create a data loader for batch training
train_loader = torch.utils.data.DataLoader(Train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(Test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
# Neural Model
class NeuralNet0(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(81, 128)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(128, 256)
        self.l3 = nn.Linear(256, 512)
        self.l4 = nn.Linear(512, 13)
        
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
model0 = NeuralNet()
num_epochs = 100
learning_rate = 0.01
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model0.parameters(), lr = learning_rate)
#writer.add_graph(model, samples.reshape(-1, 28*28))
#writer.add_graph(model, examples)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)
# Training loop
n_total_step = len(train_loader)
for epoch in range(num_epochs):
    # Total loss on all training sample
    total_loss = 0
    # Total accuracy on all training sample
    total_accuracy = 0
    for i, (data, labels) in enumerate(train_loader):
        # Batch size: 1000000
        # Total batch: 341 
        # 1000000, 81
        labels = labels.to(device, dtype = torch.long) # Change labels to long data type for cross entropy
        data = data.to(device)
        
        # forward 
        outputs = model0(data)
        loss = criterion(outputs, labels)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Batch loss
        batch_loss = loss.item()
        
        # Batch accuracy calculate
        _, predicted = torch.max(outputs, 1) # Get prediction on batch (shape: batch_size)
        batch_correct_predicted = (predicted == labels).sum().item() # Get number of true prediction on batch
        batch_accuracy = batch_correct_predicted/batch_size*100 # Get prediction accuracy 
        
        # Adding batch loss to total loss
        total_loss += batch_loss
        # Adding batch accuracy to total accuracy
        total_accuracy += batch_accuracy
        print(f'epoch {epoch + 1}/{num_epochs}, step [{i+1}/{n_total_step}], loss = {batch_loss:.3f}, acc = {batch_accuracy}')
    scheduler.step()
    print('Epoch: [{}/{}], Loss: {:.3f}, Accuracy: {:.3f}'.format(epoch + 1, num_epochs, total_loss/n_total_step, total_accuracy/n_total_step))
    PATH = '/home/jun/CICIDS2019/saved_model/lr0.0005model' + str(epoch+1) + 'epoch.pth'
    torch.save(model.state_dict(), PATH)


epoch 1/100, step [1/35], loss = 1616964.250, acc = 7.861
epoch 1/100, step [2/35], loss = 18479442.000, acc = 10.9936
epoch 1/100, step [3/35], loss = 2396613.750, acc = 23.03
epoch 1/100, step [4/35], loss = 392116.938, acc = 31.4328
epoch 1/100, step [5/35], loss = 118942.055, acc = 13.0602
epoch 1/100, step [6/35], loss = 46043.098, acc = 14.7407
epoch 1/100, step [7/35], loss = 12960.219, acc = 19.0364
epoch 1/100, step [8/35], loss = 939.276, acc = 9.8412
epoch 1/100, step [9/35], loss = 80.231, acc = 7.4739
epoch 1/100, step [10/35], loss = 15.849, acc = 12.1159
epoch 1/100, step [11/35], loss = 9.274, acc = 30.7457
epoch 1/100, step [12/35], loss = 5.992, acc = 38.9699
epoch 1/100, step [13/35], loss = 4.974, acc = 40.0546
epoch 1/100, step [14/35], loss = 4.049, acc = 40.054899999999996
epoch 1/100, step [15/35], loss = 3.805, acc = 40.0996
epoch 1/100, step [16/35], loss = 2.786, acc = 40.048
epoch 1/100, step [17/35], loss = 2.693, acc = 40.0648
epoch 1/100, step [18/35], lo